# Example 2: PennyLane VQE calculation on water

Unlike the previous case, this example showcases the use of `qc2` in combination with `PennyLane`, focusing once again on water as our target system. All circuit evaluations are conducted using the `default.qubit` state simulator device, which provides exact expectation values.

### Import required packages

In [1]:
import subprocess

# ASE molecule object
from ase.build import molecule

# Import mapper from Qiskit
from qiskit_nature.second_q.mappers import JordanWignerMapper

# PennyLane-related packages
import pennylane as qml
from pennylane import numpy as np

# ignore package import warnings
import warnings
warnings.filterwarnings('ignore')

# qc2 packages
from qc2.ase import Psi4
from qc2.data import qc2Data

### Instantiate `qc2Data` class and run qc2-ASE calculator

In [2]:
# set Atoms object
mol = molecule('H2O')

# instantiate qc2Data class
qc2data = qc2Data(
    molecule=mol,
    filename='h2o.fcidump',
    schema='fcidump'
)

# specify the qchem calculator
qc2data.molecule.calc = Psi4(method='hf', basis='sto-3g')

# run calculation and save qchem data in the fcidump file
qc2data.run()

  Python driver attempted to set threads to 1.
  Psi4 was compiled without OpenMP, setting threads to 1.
  Python driver attempted to set threads to 1.
  Psi4 was compiled without OpenMP, setting threads to 1.
* Reference energy (Hartree): -74.96449224628003
* Saving qchem data in h2o.fcidump



This generates a `h2o.fcidump` file. Like before, let's check its contents via the shell command:

In [3]:
!cat h2o.fcidump

&FCI
NORB=7,
NELEC=10,
MS2=0,
UHF=.FALSE.,
ORBSYM=1,1,1,1,1,1,1,
ISYM=1,
&END
  4.74470140637415838114E+00   1   1   1   1
  4.17377194997864575665E-01   1   1   2   1
  1.00521128156228156669E+00   1   1   2   2
  7.97099201188999018086E-01   1   1   3   3
  1.83066924455251772708E-01   1   1   4   1
  1.30763348764042580674E-01   1   1   4   2
  9.94696346386588770017E-01   1   1   4   4
  1.11533812966665379918E+00   1   1   5   5
  2.34683004432547331897E-01   1   1   6   1
  3.05430189901770843264E-01   1   1   6   2
 -2.23634622194125970429E-01   1   1   6   4
  8.02073478270340034513E-01   1   1   6   6
  3.62763332124976689563E-01   1   1   7   3
  8.65921285785869687679E-01   1   1   7   7
  4.17377194997864464643E-01   2   1   1   1
  5.83460719379530270978E-02   2   1   2   1
  1.31241428026074979235E-02   2   1   2   2
  4.42216423826163797933E-03   2   1   3   3
  2.25571363980967692919E-02   2   1   4   1
  9.15632679593813073038E-03   2   1   4   2
  1.338681787701838897

### Building up the molecular qubit Hamiltonian

In [4]:
# define activate space
n_active_electrons = (2, 2)    # => (n_alpha, n_beta)
n_active_spatial_orbitals = 3  # => active orbitals comprise of only 2px, 2py and 2pz of oxigen

# define the type of fermionic-to-qubit transformation
mapper = JordanWignerMapper()

# set up qubit Hamiltonian and core energy based on given activate space
e_core, qubit_op = qc2data.get_qubit_hamiltonian(
    n_active_electrons,
    n_active_spatial_orbitals,
    mapper,
    format='pennylane'
)

print(f'* Core energy (hartree): {e_core}')
print(f'* Molecular Hamiltonian in qubit representation: \n {qubit_op}')

* Core energy (hartree): -68.82710474045477
* Molecular Hamiltonian in qubit representation: 
   (-4.519575071756138) [I0]
+ (0.09467741944601746) [Z5]
+ (0.09467741944601749) [Z4]
+ (0.4799526042541725) [Z2]
+ (0.4799526042541727) [Z3]
+ (0.5128581791490245) [Z0]
+ (0.5128581791490247) [Z1]
+ (0.030052331652157403) [Y0 Y4]
+ (0.030052331652157403) [X0 X4]
+ (0.030052331652157403) [Y1 Y5]
+ (0.030052331652157403) [X1 X5]
+ (0.11952117163005273) [Z0 Z4]
+ (0.11952117163005273) [Z1 Z5]
+ (0.13727892451332155) [Z0 Z5]
+ (0.13727892451332155) [Z1 Z4]
+ (0.1375753792278971) [Z2 Z4]
+ (0.1375753792278971) [Z3 Z5]
+ (0.14713876142379662) [Z2 Z5]
+ (0.14713876142379662) [Z3 Z4]
+ (0.1491669147074381) [Z4 Z5]
+ (0.16764521474693636) [Z0 Z2]
+ (0.16764521474693636) [Z1 Z3]
+ (0.1815074350509799) [Z0 Z3]
+ (0.1815074350509799) [Z1 Z2]
+ (0.19405820482054056) [Z0 Z1]
+ (0.220039773343761) [Z2 Z3]
+ (0.07925281259735449) [Y1 Z3 Y5]
+ (0.07925281259735449) [X1 Z3 X5]
+ (0.07925281259735456) [Y0 Z2 Y

### Set up reference state, qnode and ansatz

In [5]:
qubits = 2 * n_active_spatial_orbitals
electrons = sum(n_active_electrons)

# Define the HF state
hf_state = qml.qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qml.qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

# Define the device
dev = qml.device("default.qubit", wires=qubits)

# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(qubit_op)

### Run VQE

In [6]:
# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))

# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

# Optimize the circuit parameters and compute the energy
for n in range(40):
    params, energy = optimizer.step_and_cost(
        circuit, params, wires=range(qubits), s_wires=s_wires,
        d_wires=d_wires, hf_state=hf_state)
    if n % 2 == 0:
        print("step = {:},  Correlation energy = {:.8f} Ha".format(n, energy))

print('')
print("=== PENNYLANE VQE RESULTS ===")
print(f"* Electronic ground state energy (hartree): {energy}")
print(f"* Inactive core energy (hartree): {e_core}")
print(f">>> Total ground state energy (hartree): {energy+e_core}\n")

step = 0,  Correlation energy = -6.13730008 Ha
step = 2,  Correlation energy = -6.14110949 Ha
step = 4,  Correlation energy = -6.14179393 Ha
step = 6,  Correlation energy = -6.14192575 Ha
step = 8,  Correlation energy = -6.14195527 Ha
step = 10,  Correlation energy = -6.14196374 Ha
step = 12,  Correlation energy = -6.14196694 Ha
step = 14,  Correlation energy = -6.14196841 Ha
step = 16,  Correlation energy = -6.14196915 Ha
step = 18,  Correlation energy = -6.14196954 Ha
step = 20,  Correlation energy = -6.14196975 Ha
step = 22,  Correlation energy = -6.14196986 Ha
step = 24,  Correlation energy = -6.14196992 Ha
step = 26,  Correlation energy = -6.14196995 Ha
step = 28,  Correlation energy = -6.14196997 Ha
step = 30,  Correlation energy = -6.14196998 Ha
step = 32,  Correlation energy = -6.14196999 Ha
step = 34,  Correlation energy = -6.14196999 Ha
step = 36,  Correlation energy = -6.14196999 Ha
step = 38,  Correlation energy = -6.14196999 Ha

=== PENNYLANE VQE RESULTS ===
* Electronic g

### Compare VQE result with classical qchem calculations

Once again, let's compare our VQE enery with the one obtained from Psi4 calculations alone:

In [7]:
import psi4

# Set Psi4 to run in serial mode
psi4.set_num_threads(1)

# Define the molecule in XYZ format
h2o = psi4.geometry("""
O 0.0000  0.000000000  0.1192622641
H 0.0000  0.763237638 -0.4770469398
H 0.0000 -0.763237638 -0.4770469398
symmetry c1
noreorient
nocom
""")

# Setup CAS space (Active space definition)
n_active_elec = 4    # Number of active electrons
n_active_orb = 3     # Number of active orbitals

# Set computation options
psi4.set_options({
    'basis': 'sto-3g',
    'reference': 'rhf'
})

# run reference Hartree-Fock
e_scf, wfn_scf = psi4.energy('scf', return_wfn=True)

# Perform FCI calculation
e_fci, wfn_fci = psi4.energy('fci/{}'.format(n_active_orb, n_active_elec), ref_wfn=wfn_scf, return_wfn=True)

# Set options for CASSCF
psi4.set_options({
    'icore': 1,
    'restricted_docc': 3
})

# Perform CASSCF calculation
e_casscf, wfn_casscf = psi4.energy('casscf/{}'.format(n_active_orb, n_active_elec), ref_wfn=wfn_scf, return_wfn=True)

# Print results
print('')
print("* Final CASSCF/sto-3g energy: {:.6f} hartree".format(e_casscf))
print("* Final FCI/sto-3g energy:    {:.6f} hartree".format(e_fci))


* Final CASSCF/sto-3g energy: -74.977870 hartree
* Final FCI/sto-3g energy:    -75.015429 hartree
